# Libraries

In [4]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms
import torchvision.utils as vutils

import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image 
import imageio
from einops import rearrange 
from tqdm import tqdm 

import wandb

%matplotlib inline

In [2]:
print("CUDA available:", torch.cuda.is_available())
print("torch version:", torch.__version__)


CUDA available: False
torch version: 2.6.0


# CVAE Model

### Creating the CVAE model.


In [ ]:


class CVAE(nn.Module): 
    def __init__(self, input_dim=38_804, h1_dim=1024, h2_dim=512, h3_dim=256, latent_dim=100): 
        super().__init__()

        # Encoder Layers 
        self.fc1 = nn.Linear(input_dim, h1_dim)
        self.fc2 = nn.Linear(h1_dim, h2_dim)
        self.fc3 = nn.Linear(h2_dim, h3_dim)
        self.fc41 = nn.Linear(h3_dim, latent_dim)  # mu 
        self.fc42 = nn.Linear(h3_dim, latent_dim)  # log var 

        # Decoder Layers 
        self.fc1 = nn.Linear(latent_dim, h1_dim)
        self.fc2 = nn.Linear(h1_dim, h2_dim)
        self.fc3 = nn.Linear(h2_dim, input_dim)
    
    def encode(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        '''
        The encoder portion of the CVAE. 

        Args: 
            x (Tensor): Input data of shape [batch_size, 178 x 218], flattened images.

        Returns:
            Tuple[Tensor, Tensor]: The mean and log-variance of the approximate posterior 
            distribution q(z|x), where z is the latent variable. 
        '''

        h1 = F.relu(self.fc1(x)) 
        h2 = self.fc2(h1)
        h3 = F.relu(self.fc3(h2))
        mu = self.fc41(h3)
        logvar = self.fc42(h3) 
        
        return mu, logvar

    def reparametrize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        '''
        Reparametrization trick standard in CVAE's. 

        Args: 
            mu: The mean of the approximate posterior distribution: q(z|x)
            logvar: The log-variance of the approximate posterior distribution: q(z|x)
        
        Returns: 
            z: The latent variable sampled from q(z|x) using the reparametrization trick.
            z is size [batch_size, latent_dim], in this case, latent_dim=100.
        ''' 
        
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)

        z = mu + std * eps 
        return z 

    def decode(self, z):
        pass  


